In [1]:
import findspark
findspark.init('/home/realtour/sparkTutorial/spark-2.4.7-bin-hadoop2.7')

In [ ]:
#notes
"""
how to install mysql jar
# https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java_8.0.23-1ubuntu20.10_all.deb
download and run command
sudo dpkg -i mysql-connector-java_8.0.23-1ubuntu20.10_all.deb

after installation checj in cd /usr/share/java/mysql-connector-java-8.0.23.jar
"""

In [2]:
#create spark session
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark # only run after findspark.init()

spark = SparkSession \
        .builder \
        .appName('test') \
        .master('local[*]') \
        .config("spark.driver.extraClassPath", "/usr/share/java/mysql-connector-java-8.0.23.jar") \
        .getOrCreate()


spark


In [4]:

vassel_df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/rahul").option("driver","com.mysql.jdbc.Driver").option("dbtable","customers").option("user","root").option("password","Meta@123").load()
# SubtitlesDashboard_df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/cloutcubetest").option("driver","com.mysql.jdbc.Driver").option("dbtable","SubtitlesDashboard").option("user","root").option("password","Meta@123").load()
print(vassel_df.show())

+----+----------+
|name|   address|
+----+----------+
|John|Highway 21|
|John|Highway 21|
+----+----------+

None
